## Distance Of Addresses

You are given a dataset of special locations (e.g. restaurants). The dataset features the name of the location, a special property (e.g. cuisine) and the address, which is broken down into street, street number, post code, city and country.

Furthermore you are given an other dataset of more locations. Your task is to add new locations from the second dataset to the first dataset, while avoiding the creation of duplicates. Keep in mind that one location may have multiple addresses!

In [1]:
# Basic Python Libs
import numpy as np
import pandas as pd
import itertools

# Distance Measurements
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

I'm assuming that the list provides valid locations, without any typos or missing information. The only possible problem to occur is if a location has multiple addresses. This can be the case if the location is right at an intersection. Otherwise we could simply compare all features for a possible new location with the features of the locations in the first dataset.

A fast, yet unrealiable methode would be to simply compare the post code of locations with the same name. This may work really good, but it certainly has the potential to erroneously identify a location as a duplicate. A possible example of that are multiple Lidl's with the same post code.

In order for us to avoid that error, we would need to make use of the street names. It would be really neat if we had a way to check if there is an alternative address for any given address. That would trivialise the problem.

Since I don't have that kind of information at hand, I'm going with a more practical approach. If a possible new location from the list shares all features but the street and street number with at least of the locations inside our dataset, I'll calculate the distance between the pairs of addresses using [geopy](https://pypi.org/project/geopy/). If the distance is less than a certain threshold, the location is assumed to be a duplicate.

To solve the problem, I need one function to compute the distance of two locations and one function to merge both dataset, identify duplicates and remove them.

In [2]:
def distance_addresses(str_address_1, str_address_2):
    """
    Takes two address strings as an input.
    The output is the distance between the two addresses in meters.
    """
    geolocator = Nominatim(user_agent="Oliver.Bey91@gmail.com")
    location_1 = geolocator.geocode(str_address_1)
    location_2 = geolocator.geocode(str_address_2)
    return geodesic((location_1.latitude, location_1.longitude), (location_2.latitude, location_2.longitude)).m

def datapoint_compare(feature_list, known_location_df, new_location_df, distance_threshold):
    """
    Takes four inputs. The first input is a list of features which shall be compared between two datapoints.
    The second input will be the dataset containing known locations. 
    As a third input, the function takes an other dataset with possible new locations.
    As the last input, we need to specify the threshold for the distance in meters.
    The output will be a modified version of the dataset containing all unique locations.
    """
    output_df = pd.concat([known_location_df, new_location_df]).reset_index()
    output_df = output_df.drop_duplicates() 
    output_df['address_string'] = output_df['street'] + \
                                  " " + output_df['street_number'] + \
                                  ", " + output_df['post_code'] + \
                                  " " + output_df['city'] + \
                                  ", " + output_df['country']
    unique_name_list = list(output_df['name'].unique())
    for name in unique_name_list:
        unique_post_code_list = list(output_df['post_code'][output_df['name'] == name].unique())
        for post_code in unique_post_code_list:
            unique_country_list = list(output_df['country'][(output_df['name'] == name) & 
                                                           (output_df['post_code'] == post_code)].unique())
            for country in unique_country_list:
                df = output_df[(output_df['name'] == name) &
                               (output_df['post_code'] == post_code) &
                               (output_df['country'] == country)].reset_index()
                if len(df) > 1:
                    address_combinations = list(itertools.combinations(df['address_string'].values, 2))
                    for combinations in address_combinations:
                        if distance_addresses(combinations[0], combinations[1]) < distance_threshold:
                            print("Duplicate detected!")
                            print(name + ", " + combinations[1])
                            output_df = output_df.loc[~((output_df['name'] == name) &
                                                  (output_df['post_code'] == post_code) &
                                                  (output_df['country'] == country) &
                                                  (output_df['address_string'] == combinations[1])),:]
    print("\nRemoving duplicates via address distance completed.")
    print("Final dataframe:\n")
    print(output_df)

In [3]:
feature_list = ['name', 'cuisine', 'street', 'street_number', 'post_code', 'city', 'country']

data_1 = {'name': ['Gaffel Haus Berlin - Das Kölsche Konsulat', 'Galeria Kaufhof Berlin'],
          'cuisine': ['German', 'European', ],
          'street': ['Dorotheenstraße', 'Alexanderplatz'],
          'street_number': ['65', '9'],
          'post_code': ['10117', '10178'],
          'city': ['Berlin', 'Berlin'],
          'country': ['Germany', 'Germany']
           }

data_2 = {'name': ['Sapori Italiani', 'Galeria Kaufhof Berlin'],
          'cuisine': ['Italian', 'European', ],
          'street': ['Alexanderplatz', 'Dircksenstr.'],
          'street_number': ['9', '2'],
          'post_code': ['10178', '10178'],
          'city': ['Berlin', 'Berlin'],
          'country': ['Germany', 'Germany']
           }

df_1 = pd.DataFrame(data_1, columns = feature_list)
df_2 = pd.DataFrame(data_2, columns = feature_list)

print(pd.concat([df_1, df_2]))

                                        name   cuisine           street  \
0  Gaffel Haus Berlin - Das Kölsche Konsulat    German  Dorotheenstraße   
1                     Galeria Kaufhof Berlin  European   Alexanderplatz   
0                            Sapori Italiani   Italian   Alexanderplatz   
1                     Galeria Kaufhof Berlin  European     Dircksenstr.   

  street_number post_code    city  country  
0            65     10117  Berlin  Germany  
1             9     10178  Berlin  Germany  
0             9     10178  Berlin  Germany  
1             2     10178  Berlin  Germany  


If we would simply merge our dataframes, we would end up with the above presentation. Now as you can see, there are two entries for __Galeria Kaufhof Berlin__. To create a duplicate, I simply picked a close by address for the second entry. Now I hope to remove that entry from our final list.

And since I'm also worried that the first dataframe may already contain duplicates, I will compute the distance for all possible combinations of entries with the same name, post code and country. If a pair of addresses is found with a distance smaller than the threshold, it will be removed from the dataframe.

In [4]:
datapoint_compare(feature_list, df_1, df_2, distance_threshold=200)

Duplicate detected!
Galeria Kaufhof Berlin, Dircksenstr. 2, 10178 Berlin, Germany

Removing duplicates via address distance completed.
Final dataframe:

   index                                       name   cuisine  \
0      0  Gaffel Haus Berlin - Das Kölsche Konsulat    German   
1      1                     Galeria Kaufhof Berlin  European   
2      0                            Sapori Italiani   Italian   

            street street_number post_code    city  country  \
0  Dorotheenstraße            65     10117  Berlin  Germany   
1   Alexanderplatz             9     10178  Berlin  Germany   
2   Alexanderplatz             9     10178  Berlin  Germany   

                              address_string  
0  Dorotheenstraße 65, 10117 Berlin, Germany  
1    Alexanderplatz 9, 10178 Berlin, Germany  
2    Alexanderplatz 9, 10178 Berlin, Germany  


As we can see, the entries

* Galeria Kaufhof Berlin, Alexanderplatz 9, 10178 Berlin, Germany
* Galeria Kaufhof Berlin, Dircksenstr. 2, 10178 Berlin, Germany

are within of 200 metres or less of each other. As a result, we dropped the later from our final dataset. 

As a note, the code can certainly be optimized. Furthermore, once you have a cleaned dataframe, it's not necessary to check for every possible combinations anymore. So memory usage can be reduced and computation time can be shorten.